In [6]:
import torch
import tiktoken
from torch import nn 
from torch.utils.data import Dataset, DataLoader

In [9]:
with open("C:\\Users\\Admin\\Downloads\\the-verdict.txt",encoding='utf-8') as f:
    text = f.read()

print(len(text))

20479


In [3]:
vocab_size = 50257
output_dim = 256

embedding_layer = torch.nn.Embedding(vocab_size,output_dim)

In [7]:
class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})

        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [8]:
def create_dataloader_v1(txt, batch_size=4, max_length=256, 
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):

    # Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # Create dataset
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # Create dataloader
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader

In [ ]:
max_len = 4
dataloader = create_dataloader_v1(text,batch_size=8,max_length=max_len,stride=max_len,shuffle=False)

dataiter = iter(dataloader)
inputs , target = next(dataiter)

print(inputs)


tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])


In [12]:
token_embedding = embedding_layer(inputs)
print(token_embedding.shape)

torch.Size([8, 4, 256])


In [14]:
context_len = max_len
pos_embedding_layer = torch.nn.Embedding(context_len,output_dim)

In [15]:
pos_embedding = pos_embedding_layer(torch.arange(max_len))
print(pos_embedding.shape)

torch.Size([4, 256])


In [16]:
input_embedding = token_embedding+pos_embedding
print(input_embedding.shape)

torch.Size([8, 4, 256])


In [17]:
input_embedding[1]

tensor([[ 0.4391,  2.0536, -1.1427,  ..., -0.4524,  1.5416,  2.3629],
        [-1.4344,  0.1224, -0.5388,  ..., -0.6012, -2.1421,  3.3849],
        [-0.4024, -0.5208, -0.3404,  ..., -0.4022, -1.6337,  0.5020],
        [ 0.1655, -0.2001, -1.1006,  ..., -0.2365, -0.8134,  0.7137]],
       grad_fn=<SelectBackward0>)